In [1]:
import random
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [2]:
filepath = tf.keras.utils.get_file('shakespeare.txt','https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()

In [4]:
text = text[300000:800000]
characters = sorted(set(text))
char_to_index = dict((c,i) for i, c in enumerate(characters))
index_to_char = dict((i,c) for i, c in enumerate(characters))
SEQ_LENGTH = 40

In [6]:
SEQ_LENGTH = 40
STEP_SIZE =3
sentences = []
next_characters = []

for i in range(0, len(text) - SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i : i+SEQ_LENGTH])
    next_characters.append(text[i+SEQ_LENGTH])

In [7]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)), dtype=np.bool_)
y = np.zeros((len(sentences), len(characters)), dtype=np.bool_)


In [8]:
for i, sentence in enumerate(sentences):
    for t, character in enumerate(sentence):
        x[i, t, char_to_index[character]]=1
    y[i, char_to_index[next_characters[i]]] = 1


In [11]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))
model.add(Dense(len(characters)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(x,y, batch_size= 256, epochs=8)

Epoch 1/8
 74/651 ━━━━━━━━━━━━━━━━━━━━ 1:41 176ms/step - loss: 3.1174

KeyboardInterrupt: 

In [12]:
model.save('textgenerator.keras')

In [20]:
from google.colab import files
files.download('textgenerator.keras')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
model = tf.keras.models.load_model('textgenerator.keras')

In [14]:
def sample(preds, temperature=1.0):
    preds=np.asarray(preds).astype('float64')
    preds=np.log(preds)/temperature
    exp_preds=np.exp(preds)
    preds=exp_preds/np.sum(exp_preds)
    probas=np.random.multinomial(1,preds,1)
    return np.argmax(probas)

In [15]:
def generate_text(length, temperature):
    start_index=random.randint(0,len(text)-SEQ_LENGTH-1)
    generated=''
    sentence = text[start_index: start_index+SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1,SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
            x[0,t,char_to_index[character]]=1

        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character

        sentence = sentence[1:]+ next_character

    return generated

In [16]:
!pip install flask-ngrok


In [17]:
!pip install flask pyngrok

In [18]:
!pip install flask-cors


In [19]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
ngrok.set_auth_token("2yr8882oIhAhvpimf4aT3lGAqDY_7U49dFAnibM2pnnCPHEpj")
from flask_cors import CORS
import threading

app = Flask(__name__)
CORS(app)

public_url = ngrok.connect(5000)
print(" * ngrok tunnel URL:", public_url)

@app.route("/generate", methods=["POST"])
def generate():
    try:
        data = request.get_json()
        print("Received data:", data)

        if not data:
            return jsonify({"error": "No JSON received"}), 400

        length = int(data.get("length", 100))
        temperature = float(data.get("temperature", 1.0))
        print(f"Length: {length}, Temperature: {temperature}")

        generated_text = generate_text(length, temperature)
        print("Generated text successfully")
        return jsonify({"output": generated_text})

    except Exception as e:
        print("Error occurred:", str(e))
        return jsonify({"error": str(e)}), 500

def run_app():
    app.run()

thread = threading.Thread(target=run_app)
thread.start()


 * ngrok tunnel URL: NgrokTunnel: "https://6b83-34-75-206-44.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
